In [1]:
import numpy as NP
from scipy import stats
from astropy import units as U
from astropy.coordinates import SkyCoord
from astropy.table import QTable
%matplotlib inline

In [2]:
def binned_statistic(x, values=None, statistic='mean', bins=10, range=None):

    """
    -----------------------------------------------------------------------------
    Same functionality as binned_statistic() under scipy.stats module but can 
    return reverse indices such as in IDL version of histogram. Read the 
    documentation on the two aforementioned functions.

    Inputs:

    x          [numpy vector] A sequence of values to be binned.

    values     [numpy vector] The values on which the statistic will be computed.
               This must be the same shape as x.

    statistic  [string or callable, optional] The statistic to compute (default
               is 'mean'). The following statistics are available:

               'mean'    : compute the mean of values for points within each bin.
                          Empty bins will be represented by NaN.
               'median'  : compute the median of values for points within each
                          bin. Empty bins will be represented by NaN.
               'count'   : compute the count of points within each bin. This is
                          identical to an unweighted histogram. values array is
                          not referenced.
               'sum'     : compute the sum of values for points within each bin.
                          This is identical to a weighted histogram.
               function : a user-defined function which takes a 1D array of
                          values, and outputs a single numerical statistic. This
                          function will be called on the values in each bin.
                          Empty bins will be represented by function([]), or NaN
                          if this returns an error.

    bins       [int or sequence of scalars, optional] If bins is an int, it
               defines the number of equal-width bins in the given range (10, by
               default). If bins is a sequence, it defines the bin edges,
               including the rightmost edge, allowing for non-uniform bin widths.

    range      [2-element tuple in list, optional] The lower and upper range of
               the bins. If not provided, range is simply (x.min(), x.max()).
               Values outside the range are ignored.

    reverse_indices
               [boolean] If set to True (default), returns the reverse indices
               in revind

    Outputs:

    statistic  [numpy vector] The values of the selected statistic in each bin.

    bin_edges  [numpy vector] Return the bin edges (length(statistic)+1).

    binnumber  [numpy vector] This assigns to each observation an integer that
               represents the bin in which this observation falls. Array has the
               same length as values.

    revind     [numpy vector] list of reverse indices like the IDL counterpart.
               Vector whose number of elements is the sum of the number of
               elements in the histogram, N, and the number of array elements
               included in the histogram, plus one. The subscripts of the
               original array elements falling in the ith bin, 0 <= i < N, are
               given by the expression: R(R[i] : R[i+1]), where R is the
               reverse index list. If R[i] is equal to R[i+1], no elements are
               present in the i-th bin. 

    -----------------------------------------------------------------------------
    """

    sortind = NP.argsort(x, kind='heapsort')
    if values is not None:
        stat, bin_edges, binnum = stats.binned_statistic(x[sortind], values[sortind], statistic=statistic, bins=bins, range=range)
    else:
        stat, bin_edges, binnum = stats.binned_statistic(x[sortind], x[sortind], statistic=statistic, bins=bins, range=range)
        
    revind = NP.hstack((bin_edges.size, bin_edges.size+NP.cumsum(stat.astype(int)), sortind))

    return stat, bin_edges, binnum, revind

In [3]:
qso_catalog = QTable.read('../Quasar_catalog_Banados+16_Matsuoka+19a_Matsuoka+19b_Wang+18_Wang+19_Reed+19_Yang+20.txt', format='ascii')
first_catalog = QTable.read('/u/thy009/carina2/bulk/projects/high-z-QSO/catalogs/first_14dec17.fits')
print(qso_catalog.info)
print(first_catalog.columns)

<QTable length=282>
  name    dtype 
-------- -------
     Ref   str12
 Ref-Num   int64
Ref-Name   str36
QSO-Name   str14
IAU-Name   str20
      RA   str12
     Dec   str12
Redshift float64
  M_1450 float64

<TableColumns names=('RA','DEC','SIDEPROB','FPEAK','FINT','RMS','MAJOR','MINOR','POSANG','FITTED_MAJOR','FITTED_MINOR','FITTED_POSANG','FLDNAME','NSDSS','SDSS_SEP','SDSS_MAG','SDSS_CLASS','NTMASS','TMASS_SEP','TMASS_MAG','YEAR','MJD','MJDRMS','MJDSTART','MJDSTOP')>


In [4]:
# Select the catalogs with the required entries
qso_catalog = qso_catalog[qso_catalog['Redshift']>=6.0]
first_catalog = first_catalog[first_catalog['SIDEPROB'] < 0.15]

In [5]:
# Display the QSO catalog
qso_catalog.show_in_notebook(display_length=300)

idx,Ref,Ref-Num,Ref-Name,QSO-Name,IAU-Name,RA,Dec,Redshift,M_1450
0,Banados+16,5,PSO~J002.3786+32.8702,QSO_J0009+3252,J000930.89+325212.94,00:09:30.89,32:52:12.94,6.1,-25.65
1,Banados+16,8,PSO~J006.1240+39.2219,QSO_J0024+3913,J002429.77+391318.98,00:24:29.772,+39:13:18.98,6.61,-25.95
2,Banados+16,9,PSO~J007.0273+04.9571,QSO_J0028+0457,J002806.56+045725.64,00:28:06.56,04:57:25.64,6.0008,-26.64
3,Banados+16,10,CFHQS~J0033--0125,QSO_J0033-0125,J003311.40-012524.90,00:33:11.40,-01:25:24.90,6.13,-25.14
4,Banados+16,14,PSO~J009.7355--10.4316,QSO_J0038-1025,J003856.52-102553.90,00:38:56.52,-10:25:53.90,6.0039,-26.54
5,Banados+16,11,PSO~J011.3898+09.0324,QSO_J0045+0901,J004533.57+090156.96,0:45:33.57,9:01:56.96,6.42,-26.01
6,Banados+16,15,CFHQS~J0050+3445,QSO_J0050+3445,J005006.67+344521.65,00:50:06.67,34:45:21.65,6.253,-26.7
7,Banados+16,16,CFHQS~J0055+0146,QSO_J0055+0146,J005502.91+014618.30,00:55:02.91,01:46:18.30,6.006,-24.81
8,Banados+16,17,SDSS~J0100+2802,QSO_J0100+2802,J010013.02+280225.92,01:00:13.02,28:02:25.92,6.3258,-29.14
9,Banados+16,19,VIK~J0109--3047,QSO_J0109-3047,J010953.13-304726.31,01:09:53.13,-30:47:26.31,6.7909,-25.64


In [6]:
first_coords = SkyCoord(ra=first_catalog['RA'], dec=first_catalog['DEC'], frame='icrs', equinox='J2000.0')
qso_coords = SkyCoord(ra=qso_catalog['RA'], dec=qso_catalog['Dec'], unit=(U.hourangle, U.deg))

In [7]:
iqso, ifirst, d2d, _d3d = first_coords.search_around_sky(qso_coords, 1/NP.sqrt(2.0)*U.deg)
nfirst, bin_edges, binnum, revind = binned_statistic(iqso, values=None, statistic='count', bins=NP.arange(qso_coords.size+1), range=None)
nfirst = nfirst.astype(int) # Number of FIRST sources in each of the quasar fields (within about 1/sqrt(2) deg radius)
print(nfirst)

[  0   0  76 109 114 107   0 110   0   0 104   0   0   0 133 127  88  93
 134 115   0 106  73   0  89 108   0   0   0  81   0   0   0   0   0   0
   0   0 130 135 137 135 121 119 113 102 107  98 109   0  74 111 154   0
  99 140 120 125 112 122 138 129 100 123 118 116 122 104  93 131 103 154
 114 123 100 125 101 132   0  90 124 111 104 110   0   0   0 132 136 101
 124 124 109 121 142   0   0   0 135   0  85  90   0   0 114 133  93  91
  45 155   0 125  99   0   0   0 108  83   0   0  61   0  83 136  94 141
 133  91 130 121 134  80 119 128  82 131 118  75 109 136 129 144 110 104
 101 146  86   0 118   0 142  97 121 131 125   0 117 118   0 147 101 124
 140  89 134 132   0   0   0 134]


In [8]:
for i in range(len(nfirst)):
    qso_name = qso_catalog['QSO-Name'][i]
    linetext = '#CRTFv0 CASA Region Text Format version 0'
    if nfirst[i] > 0: # Non-zero FIRST sources in the field
        ri = revind[revind[i]:revind[i+1]]
        cat_inds = ifirst[ri]
        cat_ra = first_coords[cat_inds].ra.deg
        cat_dec = first_coords[cat_inds].dec.deg
        cat_flux = first_catalog[cat_inds]['FINT'].to('mJy').value
        cat_majax = 2*0.5*first_catalog[cat_inds]['FITTED_MAJOR'].to('arcsec').value
        cat_minax = cat_majax
        cat_radius = cat_majax
        cat_posang = NP.zeros(nfirst[i])
        for j in range(nfirst[i]):
            linetext += '\ncircle [[{0:.7f}deg, {1:.7f}deg], {2:.3f}arcsec] coord=J2000, corr=[I], linewidth=2, linestyle=-, symsize=1, symthick=1, color=d9822b, font=Helvetica, fontsize=10, fontstyle=bold, usetex=false'.format(cat_ra[j], cat_dec[j], cat_radius[j])
            ## For elliptical region comment line above and uncomment line below, for circular regions uncomment line above and comment line below
            # linetext += '\nellipse [[{0:.7f}deg, {1:.7f}deg], [{2:.3f}arcsec, {3:.3f}arcsec], {4:.3f}deg] coord=J2000, corr=[I], linewidth=2, linestyle=-, symsize=1, symthick=1, color=d9822b, font=Helvetica, fontsize=10, fontstyle=bold, usetex=false'.format(cat_ra[j], cat_dec[j], cat_majax[j], cat_minax[j], cat_posang[j])
        
        fname = '/u/thy009/carina2/bulk/projects/high-z-QSO/data/region_files/FIRST/{0}-regions.txt'.format(qso_name)
        with open(fname, 'w') as fobj:
            fobj.write(linetext)        

